# Flow Parser
This is an example of how to read a Wikimedia Flow dump xml file and transform a filter of that data into a pandas dataframe. Flow - also now known as "structured discussions" does have an HTTP API, but does not allow you to go arbitrarily back in history.

In [92]:
#Download at https://dumps.wikimedia.org/frwiki/DATEHERE/
flow_dump_file ='/home/paprika/Downloads/frwiki-20181120-flow.xml'
import xml.etree.ElementTree as ET # import entire module; use alias for clarity
import sys, os
import pandas as pd
from collections import defaultdict

# note that this is the *module*'s `register_namespace()` function
ET.register_namespace("flow", "http://www.mediawiki.org/xml/flow-1.0/")

tree = ET.ElementTree() # instantiate an object of *class* `ElementTree`# xml.etree.register_namespace('', 'http://www.mediawiki.org/xml/flow-1.0/')
tree.parse(flow_dump_file)
e = tree.getroot()


In [72]:
forum_e = None
for atype in e.findall('{http://www.mediawiki.org/xml/flow-1.0/}board'):  
    if atype.attrib['title']=='Wikipédia:Forum_des_nouveaux':
        forum_e = atype

In [145]:
user_ids_level_post = defaultdict(lambda :defaultdict(int))
## the first level is the user name and the second key is responding

def update_from_post(post, level):
    global user_ids_posting
    for revisions in post.findall('{http://www.mediawiki.org/xml/flow-1.0/}revisions'):
        for revision in revisions:
#             print(revision)
            userid = revision.attrib['userid']
            user_ids_level_post[userid][level]+=1

def iter_children(post, level=0):
    update_from_post(post, level=level)
    for children in post.findall('{http://www.mediawiki.org/xml/flow-1.0/}children'):
        for child in children:
            if child.tag=='{http://www.mediawiki.org/xml/flow-1.0/}post':
                iter_children(child, level=level+1) 
        
for topic in forum_e.findall('{http://www.mediawiki.org/xml/flow-1.0/}topic'):
    for post in topic.findall('{http://www.mediawiki.org/xml/flow-1.0/}post'):
        iter_children(post)
            

In [148]:
user_post_df = pd.DataFrame.from_dict(user_ids_level_post, orient='index')

In [149]:
user_post_df.fillna(0, inplace=True)

In [150]:
user_post_df.reset_index(inplace=True)
user_post_df.rename(columns={'index':'user_id',0:'newquestion',1:'reply_depth_1',2:'reply_depth_2',3:'reply_depth_3'}, inplace=True)

In [151]:
user_post_df

,user_id,newquestion,reply_depth_1,reply_depth_2,reply_depth_3
0,0,1367.0,1925.0,63.0,3.0
1,10017,0.0,1.0,0.0,0.0
2,1002183,2.0,3.0,0.0,0.0
3,1002222,1.0,1.0,0.0,0.0
4,1005535,0.0,1.0,0.0,0.0
5,1010589,0.0,2.0,0.0,0.0
6,1014084,0.0,2.0,0.0,0.0
7,1015089,1.0,1.0,1.0,0.0
8,1016686,1.0,2.0,0.0,0.0
9,101819,0.0,12.0,0.0,0.0


In [126]:
!mkdir outputs

In [152]:
user_post_df.to_csv('outputs/fr_user_ids_posting_forum_des_nouveaux_201811280.csv')

In [1]:
!head -n 1 $flow_dump_file

^C
